In [8]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt


In [50]:
df=pd.read_csv("/content/epi_r.csv")
df
df.shape
df.describe()
df=df.drop('title', axis=1)
df.fillna(df.mean(), inplace=True)

df.head()

,rating,calories,protein,fat,sodium,#cakeweek,#wasteless,22-minute meals,3-ingredient recipes,30 days of groceries,...,yellow squash,yogurt,yonkers,yuca,zucchini,cookbooks,leftovers,snack,snack week,turkey
0,2.500,426.000000,30.000000,7.000000,559.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,4.375,403.000000,18.000000,23.000000,1439.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.750,165.000000,6.000000,7.000000,165.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5.000,6322.958017,100.160793,346.877497,6225.974895,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.125,547.000000,20.000000,32.000000,452.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
features = df.drop('rating', axis=1)
target = df['rating']

In [20]:
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)


In [21]:
pca = PCA(n_components=2)
principal_components = pca.fit_transform(scaled_features)


In [22]:
pca_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])
final_df = pd.concat([pca_df, target], axis=1)
print(final_df.head())


        PC1       PC2  rating
0  1.731466 -1.706333   2.500
1  2.334868  1.092495   4.375
2  1.910902 -1.246445   3.750
3 -2.472714 -2.395451   5.000
4  0.683142 -0.683413   3.125


In [24]:
pca_df

,PC1,PC2
0,1.731466,-1.706333
1,2.334868,1.092495
2,1.910902,-1.246445
3,-2.472714,-2.395451
4,0.683142,-0.683413
...,...,...
20047,2.077530,0.842774
20048,-2.997966,-0.970702
20049,-1.190938,-1.237155
20050,2.059728,-1.167518


In [34]:
# Features (X) - usually all the columns except the target
X = final_df.drop('rating', axis=1)

# Target (Y) - the column you're predicting
Y = final_df['rating']

# Display the first few rows of X and Y
print("Features (X):")
print(X.head())

print("\nTarget (Y):")
print(Y.head())


Features (X):
        PC1       PC2
0  1.731466 -1.706333
1  2.334868  1.092495
2  1.910902 -1.246445
3 -2.472714 -2.395451
4  0.683142 -0.683413

Target (Y):
0    2.500
1    4.375
2    3.750
3    5.000
4    3.125
Name: rating, dtype: float64


In [42]:
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Display the shape of the splits
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("Y_train shape:", Y_train.shape)
print("Y_test shape:", Y_test.shape)



X_train shape: (16041, 2)
X_test shape: (4011, 2)
Y_train shape: (16041,)
Y_test shape: (4011,)


In [44]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score


In [45]:
# Initialize the Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Fit the model to the training data
rf_model.fit(X_train, Y_train)


RandomForestRegressor(random_state=42)

In [48]:
# Predict the target variable for the test set
Y_pred_rf = rf_model.predict(X_test)
print(Y_pred_rf)
print(Y_test)


[3.8125  3.175   3.6375  ... 2.7875  2.15625 3.7    ]
16323    4.375
4977     2.500
16083    2.500
14127    3.750
15496    3.750
         ...  
18729    4.375
3379     4.375
2635     3.750
16302    5.000
1458     3.750
Name: rating, Length: 4011, dtype: float64


In [47]:
# Calculate Mean Squared Error
mse_rf = mean_squared_error(Y_test, Y_pred_rf)

# Calculate R-squared
r2_rf = r2_score(Y_test, Y_pred_rf)

print(f"Random Forest Mean Squared Error: {mse_rf}")
print(f"Random Forest R-squared: {r2_rf}")


Random Forest Mean Squared Error: 1.6114181351305168
Random Forest R-squared: 0.05319331314759246


In [49]:
# Print actual vs predicted values
print("\nActual vs Predicted (Random Forest):")
for actual, predicted in zip(Y_test, Y_pred_rf):
    print(f"Actual: {actual:.3f}, Predicted: {predicted:.3f}")



Actual vs Predicted (Random Forest):
Actual: 4.375, Predicted: 3.812
Actual: 2.500, Predicted: 3.175
Actual: 2.500, Predicted: 3.638
Actual: 3.750, Predicted: 4.075
Actual: 3.750, Predicted: 4.050
Actual: 0.000, Predicted: 4.306
Actual: 5.000, Predicted: 4.650
Actual: 4.375, Predicted: 4.162
Actual: 5.000, Predicted: 3.550
Actual: 3.750, Predicted: 4.200
Actual: 4.375, Predicted: 3.731
Actual: 4.375, Predicted: 3.456
Actual: 0.000, Predicted: 3.475
Actual: 3.750, Predicted: 4.500
Actual: 4.375, Predicted: 4.025
Actual: 2.500, Predicted: 4.069
Actual: 5.000, Predicted: 4.000
Actual: 3.750, Predicted: 1.919
Actual: 5.000, Predicted: 4.412
Actual: 5.000, Predicted: 4.725
Actual: 5.000, Predicted: 4.500
Actual: 4.375, Predicted: 3.731
Actual: 3.750, Predicted: 3.900
Actual: 4.375, Predicted: 4.088
Actual: 0.000, Predicted: 3.844
Actual: 4.375, Predicted: 3.444
Actual: 4.375, Predicted: 3.850
Actual: 4.375, Predicted: 3.606
Actual: 3.750, Predicted: 3.956
Actual: 5.000, Predicted: 4.487
Ac